In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from enum import Enum

class ProposalType(Enum):
    GLOBAL = 'global'
    LOCAL = 'local'
    FREEFORM = 'freeform'
    ALGORITHM = 'algorithm'
    
class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'
    
from matplotlib.path import Path
%matplotlib inline

from shapely.geometry import Polygon

In [2]:
dms = dict([(sc, DataManager(stack='MD594', section=sc, segm_params_id='tSLIC200', load_mask=False)) 
            for sc in range(47, 185)])

In [3]:
for dm in dms.itervalues():
    dm.load_multiple_results(['texHist', 'spCentroids'])

In [4]:
label_examples = pickle.load(open(os.environ['GORDON_RESULT_DIR']+'/database/label_examples.pkl', 'r'))
label_position = pickle.load(open(os.environ['GORDON_RESULT_DIR']+'/database/label_position.pkl', 'r'))
label_polygon = pickle.load(open(os.environ['GORDON_RESULT_DIR']+'/database/label_polygon.pkl', 'r'))
label_texture = pickle.load(open(os.environ['GORDON_RESULT_DIR']+'/database/label_texture.pkl', 'r'))

In [ ]:
# d = dict([(sec, np.array(polygon.exterior.coords, np.int)) for sec, polygon in label_polygon['7N'].iteritems()])
# pickle.dump(d, open('/tmp/boundarCoords_7N.pkl', 'w'))

In [ ]:
# dict([(sec, np.array(polygon.exterior.coords, np.int)) for sec, polygon in label_polygon['Pn'].iteritems()])
# pickle.dump(d, open('/tmp/boundarCoords_Pn.pkl', 'w'))

In [5]:
def grow_cluster_section(sec, *args, **kwargs):
    return dms[sec].grow_cluster(*args, **kwargs)

def grow_clusters_from_sps(sec, sps):
    
    expansion_clusters_tuples = Parallel(n_jobs=16)\
    (delayed(grow_cluster_section)(sec, s, verbose=False, all_history=False, seed_weight=0,\
                                   num_sp_percentage_limit=0.05, min_size=1, min_distance=2, threshold_abs=-0.1,
                                   threshold_rel=0.02, peakedness_limit=.002, method='rc-mean',
                                   seed_dist_lim = 0.2, inter_sp_dist_lim=1.) for s in sps)

    all_seed_cluster_score_tuples = [(seed, cl, sig) for seed, peaks in enumerate(expansion_clusters_tuples) 
                                     for cl, sig in zip(*peaks)]
    all_seeds, all_clusters, all_scores = zip(*all_seed_cluster_score_tuples)

    all_clusters_unique_dict = {}
    for i, cl in enumerate(all_clusters):
        all_clusters_unique_dict[frozenset(cl)] = i

    all_unique_cluster_indices = all_clusters_unique_dict.values()
    all_unique_cluster_scores = [all_scores[i] for i in all_unique_cluster_indices]
    all_unique_cluster_indices_sorted = [all_unique_cluster_indices[i] for i in np.argsort(all_unique_cluster_scores)[::-1]]

    all_unique_tuples = [all_seed_cluster_score_tuples[i] for i in all_unique_cluster_indices_sorted]

    return all_unique_tuples

In [6]:
def compute_cluster_coherence_score(sec, cluster, verbose=False):
    
    if len(cluster) > 1:
        cluster_avg = dms[sec].texton_hists[cluster].mean(axis=0)
        ds = np.squeeze(chi2s([cluster_avg], dms[sec].texton_hists[list(cluster)]))
        var = ds.mean()
    else:
        var = 0
    
    return var

# def compute_cluster_significance_score(sec, *args, **kwargs):
#     return dms[sec].compute_cluster_score(*args, **kwargs)[0]

In [7]:
coherence_limit = .25
area_limit = 60000
nonoverlapping_area_limit = 2.
bg_texton = 3
bg_texton_percentage = .2
significance_limit = 0.05
consensus_limit = -20

In [8]:
def compute_proposal_properties(seeded_proposals, sec):
    
    dm = dms[sec]
    
    dm.load_multiple_results(['texHist', 'spAreas', 'spCentroids'])

    properties = []
    for seed, sps, sig in seeded_proposals:
        props = {'seed': seed, 'sps': sps, 'sig': sig}
        props['coherence'] = compute_cluster_coherence_score(sec, sps)
        props['hist'] = dm.texton_hists[sps].mean(axis=0)
        nz_hist = props['hist']
        props['entropy'] = np.nan_to_num(-np.sum(nz_hist*np.log(nz_hist)))
        props['centroid'] = dm.sp_centroids[cl, ::-1].mean(axis=0)
        props['area'] = dm.sp_areas[sps].sum()
        
        properties.append(props)
        
    return properties

In [ ]:
from operator import attrgetter

In [9]:
def sort_clusters(sec, all_unique_tuples, label, sec2):
    
    dm = dms[sec]
    
    seeds, clusters, sigs = zip(*all_unique_tuples)
    
#     props = compute_proposal_properties(all_unique_tuples, sec)  
#     cluster_areas = map(attrgetter('area'), props)
#     cluster_centers = map(attrgetter('centroid'), props)
    
    tex_dists = chi2s(label_texture[label], [dm.texton_hists[sps].mean(axis=0) for sps in clusters])
    
#     remaining_cluster_indices_sortedByTexture = [remaining_cluster_indices[j] for j in np.argsort(tex_dists)]
      
    polygons = [Polygon(dm.vertices_from_dedges(dm.find_boundary_dedges_ordered(cl))) 
                for cl in clusters]

    polygon_overlaps = []
    for p in polygons:
        try:
#             polygon_overlaps.append(label_polygon[label][sec2].intersection(p).area)
            polygon_overlaps.append(float(label_polygon[label][sec2].intersection(p).area)/label_polygon[label][sec2].union(p).area)
        except:
#             print list(p.exterior.coords)
            polygon_overlaps.append(0)
    
#     rank = np.argsort(.3*scores_to_vote(polygon_overlaps) + .7*scores_to_vote(-tex_dists))[::-1]
    rank = np.argsort(.5*scores_to_vote(polygon_overlaps) + .5*scores_to_vote(-tex_dists))[::-1]

    all_remaining_clusters_sorted = [clusters[i] for i in rank]

#     remaining_cluster_indices_sortedByOverlap = [remaining_cluster_indices[j] for j in np.argsort(polygon_overlaps)[::-1]]
    
#     all_remaining_clusters_sortedByTexture = [all_unique_clusters[i] for i in remaining_cluster_indices_sortedByTexture]

#     all_remaining_clusters_sortedByOverlap = [all_unique_clusters[i] for i in remaining_cluster_indices_sortedByOverlap]
    
#     return all_remaining_clusters_sortedByTexture
#     return all_remaining_clusters_sortedByOverlap
    return all_remaining_clusters_sorted

In [10]:
section = 65

In [ ]:
ks = np.array(label_position[label].keys())
ds = ks - section

next_labeled_section = ks[ds >= 0][0]
prev_labeled_section = ks[ds <= 0][0]

print 'prev_labeled_section', prev_labeled_section, 'next_labeled_section', next_labeled_section

if abs(section - prev_labeled_section) < abs(section - next_labeled_section):
    v1,v2,s1,s2,c0 = label_position[label][prev_labeled_section]
else:
    v1,v2,s1,s2,c0 = label_position[label][next_labeled_section]

angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
ell_vertices = cv2.ellipse2Poly(tuple(c0.astype(np.int)), (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, 10)

sps = np.where([Path(ell_vertices).contains_point(s) for s in dms[section].sp_centroids[:,::-1]])[0]

print '%d sps to look at\n' % len(sps)

all_unique_tuples = grow_clusters_from_sps(section, sps)

if abs(section - prev_labeled_section) < abs(section - next_labeled_section):
    clusters_sorted = sort_clusters(section, all_unique_tuples, label, prev_labeled_section)
else:
    clusters_sorted = sort_clusters(section, all_unique_tuples, label, next_labeled_section)

dm = dms[section]

cs = dm.sp_centroids[clusters_sorted[0]]
tight_ymin, tight_xmin = cs.min(axis=0).astype(np.int) - 300

close_up = img_as_ubyte(dm.visualize_cluster(clusters_sorted[0], tight=True))

angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
cv2.ellipse(close_up, tuple(c0.astype(np.int)-(tight_xmin, tight_ymin)), 
        (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, (0,255,0), 10)

plt.imshow(close_up);
# imsave('/tmp/close_up%d_%s.jpg'%(section, label), close_up)

In [ ]:
section = 140

In [ ]:
search_area_ellipses = {}
detected_clusters = {}

for label in ['Pn', '7n', '7N']:

    ks = np.array(sorted(label_position[label].keys()))
    ds = ks - section

    next_labeled_section = ks[ds >= 0][0]
    prev_labeled_section = ks[ds <= 0][-1]

    print 'prev_labeled_section', prev_labeled_section, 'next_labeled_section', next_labeled_section

    if abs(section - prev_labeled_section) < abs(section - next_labeled_section):
        v1,v2,s1,s2,c0 = label_position[label][prev_labeled_section]
    else:
        v1,v2,s1,s2,c0 = label_position[label][next_labeled_section]

    search_area_ellipses[label] = (v1,v2,s1,s2,c0)
        
    angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
    ell_vertices = cv2.ellipse2Poly(tuple(c0.astype(np.int)), (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, 10)

    sps = np.where([Path(ell_vertices).contains_point(s) for s in dms[section].sp_centroids[:,::-1]])[0]

    print '%d sps to look at\n' % len(sps)

    all_unique_tuples = grow_clusters_from_sps(section, sps)
    
    if abs(section - prev_labeled_section) < abs(section - next_labeled_section):
        clusters_sorted = sort_clusters(section, all_unique_tuples, label, prev_labeled_section)
    else:
        clusters_sorted = sort_clusters(section, all_unique_tuples, label, next_labeled_section)
        
    dm = dms[section]

    close_up = img_as_ubyte(dm.visualize_cluster(clusters_sorted[0], tight=True))
    imsave('/tmp/closeup_%04d_%s.jpg'%(section, label), close_up)

    detected_clusters[label] = clusters_sorted[0]

In [ ]:
detected_cluster_edges = dict([(l, dm.find_boundary_dedges_ordered(cl)) for l, cl in detected_clusters.iteritems()])

In [ ]:
atlas = img_as_ubyte(dm.visualize_edge_sets(detected_cluster_edges.values(), bg='originalImage'))

for label in ['Pn', '7n', '7N']:
    v1,v2,s1,s2,c0 = search_area_ellipses[label]

    angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
    cv2.ellipse(atlas, tuple(c0.astype(np.int)-(dm.xmin, dm.ymin)), 
                (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, (0,255,0), 10)
    
    cy, cx = dm.sp_centroids[detected_clusters[label]].mean(axis=0)
    
    cv2.putText(atlas, label, (int(cx)-dm.xmin, int(cy)-dm.ymin), cv2.FONT_HERSHEY_DUPLEX,
                            3, ((255,0,0)), 3)

imsave('/tmp/labeled_%04d_allLandmarks.jpg'%section, atlas)

In [ ]:
pickle.dump(search_area_ellipses, open('/tmp/search_area_ellipses_%04d.pkl'%section, 'w'))
pickle.dump(detected_clusters, open('/tmp/detected_clusters_%04d.pkl'%section, 'w'))

In [ ]:
# for section in range(139, 144):

#     dm = dms[section]

# #     search_area_ellipses = pickle.load(open('/tmp/search_area_ellipses_%04d.pkl'%section, 'r'))
#     detected_clusters = pickle.load(open('/tmp/detected_clusters_%04d.pkl'%section, 'r'))

# #     for label, cl in detected_clusters.iteritems():
    
# #         edges = dm.find_boundary_dedges_ordered(cl)
# #         close_up = img_as_ubyte(dm.visualize_edge_set(edges, tight=True, bg='originalImage', linewidth=20))
# #         imsave('/tmp/closeup_%04d_%s.jpg'%(section, label), close_up)
        
        
#     detected_cluster_edges = dict([(l, dm.find_boundary_dedges_ordered(cl)) 
#                                    for l, cl in detected_clusters.iteritems()])

#     atlas = img_as_ubyte(dm.visualize_edge_sets(detected_cluster_edges.values(), bg='originalImage',
#                                                linewidth=20))

#     for label in ['Pn', '7n', '7N']:
#         v1,v2,s1,s2,c0 = search_area_ellipses[label]
#         angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
#         cv2.ellipse(atlas, tuple(c0.astype(np.int)-(dm.xmin, dm.ymin)), 
#                     (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, (0,255,0), 10)

#         cy, cx = dm.sp_centroids[detected_clusters[label]].mean(axis=0)

#         cv2.putText(atlas, label, (int(cx)-dm.xmin, int(cy)-dm.ymin), cv2.FONT_HERSHEY_DUPLEX,
#                                 3, ((255,0,0)), 3)

#     imsave('/tmp/labeled_%04d_allLandmarks.jpg'%section, atlas)

In [21]:
# for section in range(139, 144):
for section in [65]:
    
    search_area_ellipses = {}
    detected_clusters = {}

#     for label in ['Pn', '7n', '7N']:
    for label in ['PrVd']:

        ks = np.array(sorted(label_position[label].keys()))

        ds = ks - section
            
        if all(ds>=0):
            next_labeled_section = ks[ds >= 0][0]
            v1,v2,s1,s2,c0 = label_position[label][next_labeled_section]
        elif all(ds<=0):
            prev_labeled_section = ks[ds <= 0][-1]
            v1,v2,s1,s2,c0 = label_position[label][prev_labeled_section]
        else:
            next_labeled_section = ks[ds >= 0][0]
            prev_labeled_section = ks[ds <= 0][-1]

            print 'prev_labeled_section', prev_labeled_section, 'next_labeled_section', next_labeled_section

            if abs(section - prev_labeled_section) < abs(section - next_labeled_section):
                v1,v2,s1,s2,c0 = label_position[label][prev_labeled_section]
            else:
                v1,v2,s1,s2,c0 = label_position[label][next_labeled_section]
        
        search_area_ellipses[label] = (v1,v2,s1,s2,c0)

        angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
        ell_vertices = cv2.ellipse2Poly(tuple(c0.astype(np.int)), (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, 10)

        sps = np.where([Path(ell_vertices).contains_point(s) for s in dms[section].sp_centroids[:,::-1]])[0]

        print '%d sps to look at\n' % len(sps)

        all_unique_tuples = grow_clusters_from_sps(section, sps)

        if all(ds<=0) or abs(section - prev_labeled_section) < abs(section - next_labeled_section):
            clusters_sorted = sort_clusters(section, all_unique_tuples, label, prev_labeled_section)
        else:
            clusters_sorted = sort_clusters(section, all_unique_tuples, label, next_labeled_section)

        dm = dms[section]

        detected_clusters[label] = clusters_sorted[0]
                
        close_up = img_as_ubyte(dm.visualize_cluster(clusters_sorted[0], tight=True, bg='originalImage'))
        imsave('/tmp/closeup_%04d_%s.jpg'%(section, label), close_up)


    detected_cluster_edges = dict([(l, dm.find_boundary_dedges_ordered(cl)) for l, cl in detected_clusters.iteritems()])

    atlas = img_as_ubyte(dm.visualize_edge_sets(detected_cluster_edges.values(), bg='originalImage'))
    
#     for label in ['Pn', '7n', '7N']:
    for label in ['PrVd']:
        v1,v2,s1,s2,c0 = search_area_ellipses[label]

#         angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
#         cv2.ellipse(atlas, tuple(c0.astype(np.int)-(dm.xmin, dm.ymin)), 
#                     (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, (0,255,0), 10)

        cy, cx = dm.sp_centroids[detected_clusters[label]].mean(axis=0)

        cv2.putText(atlas, label, (int(cx)-dm.xmin, int(cy)-dm.ymin), cv2.FONT_HERSHEY_DUPLEX,
                                3, ((255,0,0)), 3)

    imsave('/tmp/labeled_%04d_allLandmarks.jpg'%section, atlas)

    pickle.dump(search_area_ellipses, open('/tmp/search_area_ellipses_%04d.pkl'%section, 'w'))
    pickle.dump(detected_clusters, open('/tmp/detected_clusters_%04d.pkl'%section, 'w'))

74 sps to look at



TopologyException: Input geom 1 is invalid: Self-intersection at or near point 4847.4438789934975 1693.6632739609838 at 4847.4438789934975 1693.6632739609838
69
25
198
152
153
144
24
80
140
22
137
190
23
85
188
82
222
352
264
268
312
297
348
285
298
224
229
330
255
251
322
202
355
442
282
479
421
418
481
414
401
356
393
396
461
363
440
339
463
576
425
606
544
564
658
527
510
466
580
523
599
467
539
447
604
646
534
660
727
655
711
585
656
720
ERROR:shapely.geos:TopologyException: Input geom 1 is invalid: Self-intersection at or near point 4847.4438789934975 1693.6632739609838 at 4847.4438789934975 1693.6632739609838
Self-intersection at or near point 4847.4438789934975 1693.6632739609838
TopologyException: Input geom 1 is invalid: Self-intersection at or near point 6011.3036830841111 3159.139042370155 at 6011.3036830841111 3159.139042370155
ERROR:shapely.geos:TopologyException: Input geom 1 is invalid: Self-intersection at or near point 6011.3036830841111 3159.139042370155 at 6011.30368

82 terminate due to seed_dist exceeds threshold 0.20126599598
69 terminate due to seed_dist exceeds threshold 0.204327844472
264 terminate due to seed_dist exceeds threshold 0.2025313339
80 terminate due to over-size
144 terminate due to over-size
140 terminate due to over-size
188 terminate due to over-size
190 terminate due to over-size
24 terminate due to over-size
25 terminate due to over-size
255 terminate due to over-size
268 terminate due to over-size
393 terminate due to over-size
467 terminate due to over-size
544 terminate due to over-size
585 terminate due to over-size
447222534285418510523348442599606655727 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
 terminate due to over-size
6464637205

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
